In [19]:
import sympy as sp
from sympy import var, Matrix, Eq, solve
sp.init_printing()
import numpy as np

In [12]:
var('''Rxy, cxy, sxy,
    Rxz, cxz, sxz,
    Rxw, cxw, sxw,
    Ryz, cyz, syz,
    Ryw, cyw, syw, 
    Rzw, czw, szw''')

Rxy = Matrix([[  cxy,  -sxy, 0.0, 0.0],
              [  sxy,   cxy, 0.0, 0.0],
              [0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])

Rxz = Matrix([[  cxz, 0.0,  -sxz, 0.0],
           [0.0, 1.0, 0.0, 0.0],
           [  sxz, 0.0,   cxz, 0.0],
           [0.0, 0.0, 0.0, 1.0]])

Rxw = Matrix([[  cxw, 0.0, 0.0,  -sxw],
           [0.0, 1.0, 0.0, 0.0],
           [0.0, 0.0, 1.0, 0.0],
           [  sxw, 0.0, 0.0,  cxw]])

Ryz = Matrix([[1.0, 0.0, 0.0, 0.0],
           [0.0,   cyz,  -syz, 0.0],
           [0.0,   syz,   cyz, 0.0],
           [0.0, 0.0, 0.0, 1.0]])

Ryw = Matrix([[1.0, 0.0, 0.0, 0.0],
           [0.0,   cyw, 0.0,  -syw],
           [0.0, 0.0, 1.0, 0.0],
           [0.0,   syw, 0.0,  cyw]])

Rzw = Matrix([[1.0, 0.0, 0.0, 0.0],
           [0.0, 1.0, 0.0, 0.0],
           [0.0, 0.0,   czw,  -szw],
           [0.0, 0.0,   szw,  czw]])

In [13]:
M = Rxy.multiply(Rxz).multiply(Rxw).multiply(Ryz).multiply(Ryw).multiply(Rzw)

In [14]:
M

⎡1.0⋅cxw⋅cxy⋅cxz  -1.0⋅cxy⋅cxz⋅sxw⋅syw + 1.0⋅cyw⋅(-1.0⋅cxy⋅sxz⋅syz - 1.0⋅cyz⋅s
⎢                                                                             
⎢1.0⋅cxw⋅cxz⋅sxy  -1.0⋅cxz⋅sxw⋅sxy⋅syw + 1.0⋅cyw⋅(1.0⋅cxy⋅cyz - 1.0⋅sxy⋅sxz⋅sy
⎢                                                                             
⎢  1.0⋅cxw⋅sxz                   1.0⋅cxz⋅cyw⋅syz - 1.0⋅sxw⋅sxz⋅syw            
⎢                                                                             
⎣    1.0⋅sxw                                1.0⋅cxw⋅syw                       

xy)  czw⋅(-1.0⋅cxy⋅cyz⋅sxz + 1.0⋅sxy⋅syz) + szw⋅(-1.0⋅cxy⋅cxz⋅cyw⋅sxw - syw⋅(-
                                                                              
z)   czw⋅(-1.0⋅cxy⋅syz - 1.0⋅cyz⋅sxy⋅sxz) + szw⋅(-1.0⋅cxz⋅cyw⋅sxw⋅sxy - syw⋅(1
                                                                              
                            1.0⋅cxz⋅cyz⋅czw + szw⋅(-1.0⋅cxz⋅syw⋅syz - 1.0⋅cyw⋅
                                                   

In [15]:
path = '../results/alignment/trajectories/'
#path = 'optML_miniproject/results/alignment/trajectories/'
file = np.load(path+'0.npy')

In [16]:
def euler6tomarix4d(a_R):
    """ Convert 6D vector containing angles to 4D rotation matrix

    a_R: tf.tensor/np.ndarray
        Vector of shape (6,)

    Returns
    -------
    R: tf.tensor/np.ndarray
        4x4 Rotation matrix corresponding to these 6 angles of rotations
    """
    xy, xz, xw, yz, yw, zw = a_R

    cxy = np.cos(xy)
    cxz = np.cos(xz)
    cxw = np.cos(xw)
    cyz = np.cos(yz)
    cyw = np.cos(yw)
    czw = np.cos(zw)

    sxy = np.sin(xy)
    sxz = np.sin(xz)
    sxw = np.sin(xw)
    syz = np.sin(yz)
    syw = np.sin(yw)
    szw = np.sin(zw)

    # Note: wasn't able to create it as simple as np.ndarrays...
    Rxy = np.array([[  cxy,  -sxy, 0.0, 0.0],
              [  sxy,   cxy, 0.0, 0.0],
              [0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])

    Rxz = np.array([[  cxz, 0.0,  -sxz, 0.0],
               [0.0, 1.0, 0.0, 0.0],
               [  sxz, 0.0,   cxz, 0.0],
               [0.0, 0.0, 0.0, 1.0]])

    Rxw = np.array([[  cxw, 0.0, 0.0,  -sxw],
               [0.0, 1.0, 0.0, 0.0],
               [0.0, 0.0, 1.0, 0.0],
               [  sxw, 0.0, 0.0,  cxw]])

    Ryz = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0,   cyz,  -syz, 0.0],
               [0.0,   syz,   cyz, 0.0],
               [0.0, 0.0, 0.0, 1.0]])

    Ryw = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0,   cyw, 0.0,  -syw],
               [0.0, 0.0, 1.0, 0.0],
               [0.0,   syw, 0.0,  cyw]])

    Rzw = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0, 1.0, 0.0, 0.0],
               [0.0, 0.0,   czw,  -szw],
               [0.0, 0.0,   szw,  czw]])

    R = Rxy @ Rxz @ Rxw @ Ryz @ Ryw @ Rzw

    return R

In [60]:
R = euler6tomarix4d(file[-1,:])

In [61]:
R

array([[ 0.34439141, -0.6893094 ,  0.50239565,  0.39223172],
       [ 0.72365664,  0.46871384,  0.39482803, -0.31739443],
       [ 0.55168842, -0.35996205, -0.73005563, -0.18189554],
       [-0.23098812, -0.41903128,  0.24233904, -0.84399589]])

In [52]:
R = np.eye(4).astype(np.int)

In [63]:
M

⎡1.0⋅cxw⋅cxy⋅cxz  -1.0⋅cxy⋅cxz⋅sxw⋅syw + 1.0⋅cyw⋅(-1.0⋅cxy⋅sxz⋅syz - 1.0⋅cyz⋅s
⎢                                                                             
⎢1.0⋅cxw⋅cxz⋅sxy  -1.0⋅cxz⋅sxw⋅sxy⋅syw + 1.0⋅cyw⋅(1.0⋅cxy⋅cyz - 1.0⋅sxy⋅sxz⋅sy
⎢                                                                             
⎢  1.0⋅cxw⋅sxz                   1.0⋅cxz⋅cyw⋅syz - 1.0⋅sxw⋅sxz⋅syw            
⎢                                                                             
⎣    1.0⋅sxw                                1.0⋅cxw⋅syw                       

xy)  czw⋅(-1.0⋅cxy⋅cyz⋅sxz + 1.0⋅sxy⋅syz) + szw⋅(-1.0⋅cxy⋅cxz⋅cyw⋅sxw - syw⋅(-
                                                                              
z)   czw⋅(-1.0⋅cxy⋅syz - 1.0⋅cyz⋅sxy⋅sxz) + szw⋅(-1.0⋅cxz⋅cyw⋅sxw⋅sxy - syw⋅(1
                                                                              
                            1.0⋅cxz⋅cyz⋅czw + szw⋅(-1.0⋅cxz⋅syw⋅syz - 1.0⋅cyw⋅
                                                   

In [64]:
a = Eq(M[0,0],R[0,0])
b = Eq(M[0,1],R[0,1])
c = Eq(M[0,2],R[0,2])
d = Eq(M[0,3],R[0,3])
e = Eq(M[1,0],R[1,0])
f = Eq(M[1,1],R[1,1])
g = Eq(M[1,2],R[1,2])
h = Eq(M[1,3],R[1,3])
i = Eq(M[2,0],R[2,0])
j = Eq(M[2,1],R[2,1])
k = Eq(M[2,2],R[2,2])
l = Eq(M[2,3],R[2,3])
m = Eq(M[3,0],R[3,0])
n = Eq(M[3,1],R[3,1])
o = Eq(M[3,2],R[3,2])
p = Eq(M[3,3],R[3,3])


solve([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p],(cxy, sxy,cxz, sxz,cxw, sxw,cyz, syz,cyw, syw, czw, szw), simplify=False, rational=False)

[]